# Test base/niagads/validate_medatata

In [45]:
import subprocess
import json

def pretty_print(resultStr):
    resultJson = json.loads(resultStr.replace('running\n', '').replace('\\"', ''))
    print(json.dumps(resultJson, indent=4))

## script test `../../bases/niagads/validate_metadata/core.py`

In [46]:
# usage 
CMD_ROOT = ['python', '../../bases/niagads/validate_metadata/core.py']
result = subprocess.run(CMD_ROOT + ['--help'], capture_output=True, text=True)
print(result.stderr)
print(result.stdout)


running
usage: core.py [-h] [--template TEMPLATE] --metadataFileType
               {BIOSOURCE_PROPERTIES,FILE_MANIFEST} [--log] [--failOnError]
               [--schemaDir SCHEMADIR]
               [--metadataFilePrefix METADATAFILEPREFIX]
               [--metadataFile METADATAFILE] [--schemaFile SCHEMAFILE]
               [--idField IDFIELD]

validate_metadata This script allows the user validate a sample or file
manifest metadata file arranged in tabular format (field names in columns,
values in rows) against a JSON-Schema file. Results are piped to STDOUT unless
`--log` option is specified. This tool accepts tab separated value files
(.tab) as well as excel (.xls, .xlsx) files. This file can also be imported as
a module and contains the following functions / tyes: * MetadataValidatorType
- enum of types of expected metadata files * initialize_validator - returns an
initialized BiosourcePropertiesValidator or FileManifestValidator *
get_templated_schema_file - builds schema file n

In [47]:
# file manifest test
schemaFile = './schemas/file_manifest.json'
metadataFile = './examples/test_file_manifest.tab'
cmd = ['python', '../bases/niagads/validate_metadata/core.py']
args = ['--metadataFileType', 'file_manifest', '--schemaFile', schemaFile, '--metadataFile', metadataFile]
result = subprocess.run(CMD_ROOT + args, capture_output=True, text=True)
print(result.stderr)
print(result.stdout)



running
[]



In [48]:
# file manifest with errors te
schemaFile = './schemas/file_manifest.json'
metadataFile = './examples/test_file_manifest_with_errors.tab'
cmd = ['python', '../bases/niagads/validate_metadata/core.py']
args = ['--metadataFileType', 'file_manifest', '--schemaFile', schemaFile, '--metadataFile', metadataFile]
result = subprocess.run(CMD_ROOT + args, capture_output=True, text=True)
print(result.stderr)
pretty_print(result.stdout)




[
    {
        "1": [
            "'4f08d67f64422' does not match '^[a-fA-F0-9]{32}$'",
            "unexpected value for `sample_id`; field is not relevant for this record; please set to an empty string in a metadata .tab/.xlsx file; `null` in a .json file",
            "'single-' is not one of ['single-sample', 'multi-sample', 'sample-independent', 'other']"
        ]
    },
    {
        "4": [
            "required field `sample_id` cannot be empty / null"
        ]
    }
]
